In [ ]:
import chipwhisperer as cw
scope = cw.scope()

In [ ]:
# program target:
bitstream = r"V:\hardware\CW305_DesignStart\CW305_DesignStart.bit"
ftarget = cw.target(scope, cw.targets.CW305, bsfile=bitstream, force=False)

In [ ]:
# set clock:
ftarget.vccint_set(1.0)
# we only need PLL1:
ftarget.pll.pll_enable_set(True)
ftarget.pll.pll_outenable_set(False, 0)
ftarget.pll.pll_outenable_set(True, 1)
ftarget.pll.pll_outenable_set(False, 2)

# run at 20 MHz:
ftarget.pll.pll_outfreq_set(20E6, 1)

In [ ]:
# ensure ADC is locked:
scope.clock.reset_adc()
assert (scope.clock.adc_locked), "ADC failed to lock"

In [ ]:
scope.gain.gain = 60
scope.gain.mode = "high"
scope.adc.samples = 20000
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "extclk_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "disabled"

In [ ]:
target = cw.target(scope)

In [ ]:
# some convenience functions:
def reset_fpga():
    # resets the full CW305 FPGA
    ftarget.fpga_write(3, [1])
    ftarget.fpga_write(3, [0])

def reset_arm_target():
    # resets only the Arm DesignStart core within the CW305 FPGA
    ftarget.fpga_write(2, [1])
    ftarget.fpga_write(2, [0])
    
def use_fpga_pll():
    # The target clock goes through a PLL (MMCM) in the FPGA before getting to the Arm DesignStart core.
    # This PLL can clean up the clock and filter glitches.
    ftarget.fpga_write(1, [1])

def bypass_fpga_pll():
    # The target clock is connected directly to the Arm DesignStart core, bypassing the PLL.
    # This can make clock glitching more effective.
    ftarget.fpga_write(1, [0])